In [1]:
import gc
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import time
from typing import Tuple

import h2o
from h2o.estimators import H2OGradientBoostingEstimator
from sklearn import metrics

import sys
sys.path.append("../utils")
from metrics import compute_recall_at4, compute_normalized_gini, compute_amex_metric
#from messaging import send_message

pd.set_option('display.max_columns', None)

H2O_server = h2o.init(port=54321, nthreads=-1)
h2o.remove_all()

Checking whether there is an H2O instance running at http://localhost:54321 .

.

.

.

.

 not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.20.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.20.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp6uwepyu_
  JVM stdout: /tmp/tmp6uwepyu_/h2o_root_started_from_python.out
  JVM stderr: /tmp/tmp6uwepyu_/h2o_root_started_from_python.err


  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ...

 successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.4
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_root_6qnlf3
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,12.26 Gb
H2O_cluster_total_cores:,20
H2O_cluster_allowed_cores:,20
H2O_cluster_status:,"locked, healthy"


In [2]:
# CONFIG PARAMS
N_REPEATS = 3
DATASET_VERSION = "06"

In [3]:
OOF_PATH = Path(f"../data/oof/h2ogbm-dsv{DATASET_VERSION}")
SUB_PATH = Path(f"../data/subs/h2ogbm-dsv{DATASET_VERSION}")
ART_PATH = Path(f"../artifacts/hg2gbm-dsv{DATASET_VERSION}")

if not OOF_PATH.exists():
    OOF_PATH.mkdir(parents=True, exist_ok=True)
if not SUB_PATH.exists():
    SUB_PATH.mkdir(parents=True, exist_ok=True)
if not ART_PATH.exists():
    ART_PATH.mkdir(parents=True, exist_ok=True)

***
## load and prepare data

In [4]:
train = pd.read_parquet(f"../data/processed/dsv{DATASET_VERSION}/train.parquet")
train_labels = pd.read_csv("../data/raw/train_labels.csv", index_col="customer_ID")

In [5]:
input_feats = train.columns.tolist()
len(input_feats)

1308

In [6]:
train = pd.merge(train, train_labels, how="inner", left_index=True, right_index=True)
del train_labels
gc.collect()

12

In [7]:
column_types = {col:"numeric" for col in input_feats}
column_types["target"] = "categorical"

***
## model training

train with repeated cross validation

In [8]:
model_params = {
    "distribution": "bernoulli",
    "seed": 2112,
    "histogram_type":"RoundRobin",
    # general params
    "learn_rate": 0.05,
    "max_depth": 5,
    'col_sample_rate': 0.2,
    'sample_rate': 0.9,
    'ntrees': 4000,
    # regularization
    'min_rows': 500,
    'max_abs_leafnode_pred':0.5,
    'min_split_improvement':1e-5,
    "nbins": 256,
}

In [9]:
def train_models(dataframe: pd.DataFrame, n_folds: int = 5,) -> tuple:
    
    models = list()
    
    # dataframe to store the oof predictions
    oof = dataframe[["target"]].copy()
    oof["pred"] = -1

    for fold in range(n_folds):
        
        print(f" training model {fold+1}/{n_folds} ".center(100, "#"))
        
        train_df = dataframe.query("fold != @fold").copy()
        valid_df = dataframe.query("fold == @fold").copy()
                                
        train_dset = h2o.H2OFrame(
            train_df,
            column_types=column_types,
        )
        valid_dset = h2o.H2OFrame(
            valid_df,
            column_types=column_types,
        )
        model = H2OGradientBoostingEstimator(**model_params)
        model.train(
            x=input_feats, 
            y="target", 
            training_frame=train_dset,
        )
        
        pred = model.predict(valid_dset).as_data_frame()["1.0"].values.flatten()
        oof.loc[valid_df.index,"pred"] = pred
        
        models.append(model)
        del train_df,valid_df,train_dset,valid_dset
        gc.collect()
    
    return models,oof

In [10]:
# implement repeated cross validation
sorted(glob("../data/processed/cv*.csv"))

['../data/processed/cv0.csv',
 '../data/processed/cv1.csv',
 '../data/processed/cv2.csv',
 '../data/processed/cv3.csv',
 '../data/processed/cv4.csv',
 '../data/processed/cv5.csv',
 '../data/processed/cv6.csv',
 '../data/processed/cv7.csv',
 '../data/processed/cv8.csv',
 '../data/processed/cv9.csv']

In [11]:
%%time 

all_models = list()
all_oof_dfs = list()

for repetition in range(N_REPEATS):
    print(f" repeated cross-validation step: {repetition+1}/{N_REPEATS} ".center(100, "#"))

    folds = pd.read_csv(f'../data/processed/cv{repetition}.csv', index_col="customer_ID")
    _train = pd.merge(train, folds, how="inner", left_index=True, right_index=True).reset_index(drop=True)
    
    tic = time.time()
    #models,oof = train_models(_train, n_folds=5)
    models,dset = train_models(_train, n_folds=5)
    tac = time.time()
    print(f"Training time: {(tac-tic)/60} min.")
    
    break
          
    # oof metrics
    print("OOF recall_at4:", compute_recall_at4(oof.target.values, oof.pred.values))
    print("OOF normalized_gini:", compute_normalized_gini(oof.target.values, oof.pred.values))
    print("OOF competition metric:", compute_amex_metric(oof.target.values, oof.pred.values))
    
    all_models.append(models)
    all_oof_dfs.append(oof)
    
    # save oof predictions
    oof.to_csv(OOF_PATH/f"oof-cv{repetition}.csv", index=False)

    del _train, folds; gc.collect()

############################### repeated cross-validation step: 1/3 ################################


######################################## training model 1/5 ########################################


Parse progress: |

█

█

...

█

█

█

████████████████████████████████████████████████████| (done) 100%


######################################## training model 2/5 ########################################


In [ ]:
results = list()

for oof in all_oof_dfaaaas:  
    r = {
        "recall_at4": compute_recall_at4(oof.target.values, oof.pred.values),
        "gini": compute_normalized_gini(oof.target.values, oof.pred.values),
        "metric": compute_amex_metric(oof.target.values, oof.pred.values),
    }
    results.append(r)

results = pd.DataFrame(results)
display(results)

print("\nmean:")
display(results.mean(axis=0))

print("\nstd:")
display(results.std(axis=0))

In [ ]:
del train
gc.collect()

***
## make predictions and submit

In [ ]:
def make_predictions(
        dataframe:pd.DataFrame, 
        input_feats:list,
        models:list,
    ) -> np.array:
    preds = list()
    for model in models:
        _dataframe = dataframe.copy()
        _dataframe_casted = h2o.H2OFrame(_dataframe, column_types=column_types)
        preds.append(model.predict(_dataframe_casted).as_data_frame()["1.0"].values.flatten())
    return np.mean(preds, axis=0)   

In [ ]:
test = pd.read_parquet(f"../data/processed/dsv{DATASET_VERSION}/test.parquet")
sub = pd.read_csv("../data/raw/sample_submission.csv")

In [ ]:
%%time

all_preds = list()

for repetition in range(N_REPEATS):
    if "prediction" in sub.columns:
        sub.drop("prediction", axis=1, inplace=True)
    if "prediction" in test.columns:
        test.drop("prediction", axis=1, inplace=True)
        
    models = all_models[repetition]
    preds = make_predictions(test, input_feats, models)
    all_preds.append(preds)
       
    test["prediction"] = preds
    sub["prediction"] = test.loc[sub.customer_ID.values,"prediction"].values
    assert sub.prediction.isna().sum() == 0
    sub.to_csv(SUB_PATH/f"submission-cv{repetition}.csv", index=False)

In [ ]:
%%time
# predict using all the trained models
if "prediction" in sub.columns:
    sub.drop("prediction", axis=1, inplace=True)
if "prediction" in test.columns:
    test.drop("prediction", axis=1, inplace=True)

test["prediction"] = np.mean(all_preds, axis=0)
sub["prediction"] = test.loc[sub.customer_ID.values,"prediction"].values
assert sub.prediction.isna().sum() == 0
sub.to_csv(SUB_PATH/f"submission-all.csv", index=False)

***